___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="CLRSWY"></p>

___

<h1 style="text-align: center;">Deep Learning<br><br>Assignment-2 (CNN)<br><br>Image Classification with CNN<br><h1>

# Task and Dataset Info

Welcome to second assignment of Deep learning lesson. Follow the instructions and complete the assignment.

**Build an image classifier with Convolutional Neural Networks for the Fashion MNIST dataset. This data set includes 10 labels of different clothing types with 28 by 28 *grayscale* images. There is a training set of 60,000 images and 10,000 test images.**

    Label	Description
    0	    T-shirt/top
    1	    Trouser
    2	    Pullover
    3	    Dress
    4	    Coat
    5	    Sandal
    6	    Shirt
    7	    Sneaker
    8	    Bag
    9	    Ankle boot

# Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

plt.rcParams["figure.figsize"] = (10,6)

sns.set_style("whitegrid")
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Set it None to display all rows in the dataframe
# pd.set_option('display.max_rows', None)

# Set it to None to display all columns in the dataframe
pd.set_option('display.max_columns', None)

In [3]:
import tensorflow as tf

tf.config.list_physical_devices("GPU")

# Recognizing and Understanding Data

**TASK 1: Run the code below to download the dataset using Keras.**

In [7]:
from tensorflow.keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [8]:
print("There are ", len(x_train), "images in the training dataset")     
print("There are ", len(x_test), "images in the test dataset")   

In [10]:
x_train[3].shape


In [11]:
x_train[3]


In [12]:
y_train[3]


In [13]:
y_train.shape


In [14]:
y_train


In [15]:
np.unique(y_train, return_counts=True)


**TASK 2: Use matplotlib to view an image from the data set. It can be any image from the data set.**

In [16]:
classes=["T-shirt/top", "Trouser","Pullover","Dress","Coat","Sandal","Shirt","Sneaker","Bag","Ankle boot"]

In [17]:
y_train[3]


In [18]:
classes[int(y_train[3])]


In [20]:
plt.figure(figsize=(20, 20))
for i in range(64):
    rand_index = int(np.random.randint(low=0, high=50000, size=1))
    plt.subplot(8, 8, i+1)
    plt.axis('off')
    plt.title(classes[int(y_train[rand_index])])
    plt.imshow(x_train[rand_index])

# Data Preprocessing

**TASK 3: Normalize the X train and X test data by dividing by the max value of the image arrays.**

In [21]:
x_train[3].min()


In [22]:
x_train[3].max()


In [23]:
X_train = x_train.astype('float32')
X_test = x_test.astype('float32')
X_train /= 255
X_test /= 255

In [24]:
X_train[3].min()


In [25]:
X_train[3].max()


In [26]:
plt.imshow(X_train[3])


In [27]:
X_train.shape, X_test.shape


**Task 4: Reshape the X arrays to include a 4 dimension of the single channel. Similar to what we did for the numbers MNIST data set.**

In [28]:
X_train.reshape(60000, 28, 28, 1).shape


In [31]:
X_train = X_train.reshape(X_train.shape[0],28, 28, 1) 
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [32]:
X_train.shape, X_test.shape


In [33]:
y_train


In [34]:
y_train.shape


**TASK 5: Convert the y_train and y_test values to be one-hot encoded for categorical analysis by Keras.**

In [35]:
from tensorflow.keras.utils import to_categorical


In [36]:
Y_train = to_categorical(y_train, 10)   
Y_test = to_categorical(y_test, 10)
Y_train.shape

In [37]:
y_train[3]


In [38]:
Y_train[3]


# Modeling

## Create the model

**TASK 5: Use Keras to create a model consisting of at least the following layers (but feel free to experiment):**

* 2D Convolutional Layer, filters=28 and kernel_size=(3,3)
* Pooling Layer where pool_size = (2,2) strides=(1,1)

* Flatten Layer
* Dense Layer (128 Neurons, but feel free to play around with this value), RELU activation

* Final Dense Layer of 10 Neurons with a softmax activation

**Then compile the model with these parameters: loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy']**

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout

In [40]:
model = Sequential()

model.add(Conv2D(filters=28, kernel_size=(3,3), input_shape=(28, 28, 1), activation='relu')) 

model.add(MaxPool2D(pool_size=(2, 2),strides = (1,1))) 

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(10, activation='softmax'))  

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [41]:
model.summary()


##  Model Training 

**TASK 6: Train/Fit the model to the x_train set by using EarlyStop. Amount of epochs is up to you.**

In [42]:
from tensorflow.keras.callbacks import EarlyStopping


In [43]:
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights = True)


In [44]:
model.fit(X_train, Y_train, batch_size=32, epochs=10, validation_data=(X_test,Y_test),callbacks=[early_stop])


**TASK 7: Plot values of metrics you used in your model.**

In [45]:
model.metrics_names


In [46]:
summary = pd.DataFrame(model.history.history)
summary.head()

In [47]:
summary.plot();


In [48]:
summary[["loss", "val_loss"]].plot();


In [49]:
summary[["accuracy", "val_accuracy"]].plot();


## Model Evaluation

**TASK 8: Show the accuracy,precision,recall,f1-score the model achieved on the x_test data set. Keep in mind, there are quite a few ways to do this, but we recommend following the same procedure we showed in the MNIST lecture.**

In [50]:
from sklearn.metrics import classification_report, confusion_matrix


In [51]:
model.evaluate(X_test, Y_test)


In [52]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [53]:
pred_prob = model.predict(X_test)


In [54]:
y_pred = np.argmax(pred_prob, axis=1)


In [55]:
y_pred


In [56]:
print(classification_report(y_test, y_pred))


In [57]:
print(confusion_matrix(y_test, y_pred))


#  Modelling-2


In [58]:
model = Sequential()

model.add(Conv2D(filters=28, kernel_size=(3,3), input_shape=(28, 28, 1), activation='relu')) 

model.add(MaxPool2D(pool_size=(2, 2),strides = (1,1))) 
model.add(Dropout(0.4))
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))  

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [59]:
model.summary()


In [61]:
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights = True)


In [62]:
model.fit(X_train, Y_train, batch_size=32, epochs=30, validation_data=(X_test,Y_test),callbacks=[early_stop])


In [63]:
model.metrics_names


In [64]:
summary = pd.DataFrame(model.history.history)
summary.head()

In [65]:
summary[["loss", "val_loss"]].plot();


In [66]:
summary[["accuracy", "val_accuracy"]].plot();


# Evaluatin on Data

In [67]:
model.evaluate(X_test, Y_test)


In [68]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [69]:
pred_prob = model.predict(X_test)


In [71]:
y_pred = np.argmax(pred_prob, axis=1)


In [72]:
y_pred


In [73]:
print(classification_report(y_test, y_pred))


In [74]:
plt.figure(figsize=(15,10))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, cmap="Blues", linewidths=.5, linecolor='black', fmt='g', annot_kws = {"size": 14})


## Prediction

In [75]:
my_image = X_test[3]


In [76]:
my_image.shape


In [77]:
plt.imshow(my_image.reshape(28,28));


In [78]:
image_prediction = model.predict(my_image.reshape(1,28,28,1))


In [79]:
image_prediction


In [80]:
np.argmax(image_prediction, axis=1)


In [81]:
result=np.argmax(image_prediction, axis=1)
result

In [82]:
classes[int(result)]


In [83]:
model.save("cnn-assign-2.h5")


# End of Assignment

___

<p style="text-align: center;"><img src="https://docs.google.com/uc?id=1lY0Uj5R04yMY3-ZppPWxqCr5pvBLYPnV" class="img-fluid" alt="CLRSWY"></p>

___